# please only run this cell if you are running the code on google colab.

In [ ]:
# Import necessary libraries
from google.colab import userdata # For accessing Hugging Face token if stored as a secret
from huggingface_hub import login # For logging into Hugging Face
from google.colab import drive # For mounting Google Drive

# Mount Google Drive - force_remount can be useful if it was mounted before
drive.mount('/content/drive', force_remount=True)

#required packages
## if you did not use google colab, please try:
## pip install -q transformers accelerate pillow datasets torchmetrics torchvision scikit-learn tqdm pandas




In [ ]:
# Install required Python packages quietly
!pip install -q transformers accelerate pillow datasets torchmetrics torchvision scikit-learn tqdm pandas

#Train:Test(val)=80:20


In [ ]:
# Standard library imports
import os
import json
import random
import numpy as np
import pandas as pd # For easier data handling

# PyTorch and related imports
import torch
import torch.nn.functional as F # For functional operations if needed
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

# Hugging Face Transformers imports
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizerFast

# Scikit-learn imports for utilities and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve as sk_roc_curve # For plotting ROC

# Other useful libraries
from tqdm.auto import tqdm # For progress bars
from PIL import Image, ImageFile # For image manipulation
import matplotlib.pyplot as plt # For plotting
import seaborn as sns # For enhanced plotting (like heatmaps)
import torchvision.transforms as T # For image transformations

# TorchMetrics for efficient and correct metric calculation
from torchmetrics import Accuracy, F1Score, Recall, AUROC, ConfusionMatrix

# Configuration to allow loading of potentially truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --- Configuration Class ---
class Config:
    # Data paths
    IMAGE_FOLDER = "/content/drive/MyDrive/RL_final_project/bdd_for_clip/data"  # CHANGE THIS TO YOUR IMAGE FOLDER PATH
    EXPLANATION_JSON = "/content/drive/MyDrive/RL_final_project/bdd_all_1a1r.json" # CHANGE THIS TO YOUR JSON PATH

    # Model configuration
    MODEL_NAME = "openai/clip-vit-base-patch32" # Pre-trained CLIP model

    # Training hyperparameters
    BATCH_SIZE = 32  # Adjust based on GPU memory
    NUM_EPOCHS = 30  # Increased epochs, use with early stopping
    LEARNING_RATE = 2e-5  # A common starting point for fine-tuning transformers with BCE
    WEIGHT_DECAY = 0.01 # Weight decay for regularization
    GRADIENT_CLIP_VAL = 1.0 # For gradient clipping to prevent exploding gradients

    # Scheduler parameters (using ReduceLROnPlateau as an example)
    PLATEAU_FACTOR = 0.2  # Factor by which the learning rate will be reduced
    PLATEAU_PATIENCE = 3  # Number of epochs with no improvement after which learning rate will be reduced

    # Data splitting and processing
    VALIDATION_SPLIT = 0.2  # Proportion of data to use for validation
    RANDOM_SEED = 42  # For reproducibility
    MAX_LENGTH = 77  # Max sequence length for CLIP tokenizer

    # Model saving
    MODEL_SAVE_DIR = "/content/drive/MyDrive/RL_final_project/clip_finetuned_models_reason/" # Directory to save models
    BEST_MODEL_NAME = "best_clip_judgement_model_reason.pth" # Name for the best model file

    # Early stopping
    EARLY_STOPPING_PATIENCE = 7  # Number of epochs to wait for improvement before stopping
    METRIC_FOR_EARLY_STOPPING = "val_auroc"  # Metric to monitor for early stopping (e.g., "val_f1", "val_accuracy")

    # Threshold for converting probabilities (after sigmoid) to binary predictions (0 or 1)
    SIMILARITY_THRESHOLD = 0.5  # Standard threshold for probabilities from sigmoid

# Instantiate configuration
config = Config()

# Create model save directory if it doesn't exist
os.makedirs(config.MODEL_SAVE_DIR, exist_ok=True)
config.MODEL_SAVE_PATH = os.path.join(config.MODEL_SAVE_DIR, config.BEST_MODEL_NAME)

# --- Utility Functions ---
def set_seed(seed):
    """Sets the seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        # The following two lines can ensure full reproducibility but might slow down training
        # torch.backends.cudnn.deterministic = True
        # torch.backends.cudnn.benchmark = False

set_seed(config.RANDOM_SEED)

# --- Hugging Face Login (Optional) ---
# Retrieves the Hugging Face token from Colab secrets if you've set one.
# This is useful if you are using private models or need to push models to the Hub.
# To set a secret: click the key icon on the left panel in Colab, add a new secret named "hf_token".
# hf_token = userdata.get("hf_token")
# if hf_token:
#     try:
#         login(token=hf_token)
#         print("Successfully logged into Hugging Face.")
#     except Exception as e:
#         print(f"Hugging Face login failed: {e}. Proceeding without login.")
# else:
#     print("Hugging Face token not found in Colab secrets. Proceeding without login.")

# --- Device Configuration ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Load CLIP Model, Processor, and Tokenizer ---
try:
    clip_model = CLIPModel.from_pretrained(config.MODEL_NAME).to(device)
    processor = CLIPProcessor.from_pretrained(config.MODEL_NAME)
    # Using CLIPTokenizerFast for potentially faster tokenization
    tokenizer = CLIPTokenizerFast.from_pretrained(config.MODEL_NAME)
    print(f"CLIP model '{config.MODEL_NAME}' and processor loaded successfully.")
except Exception as e:
    print(f"Error loading CLIP model or processor: {e}. Please ensure the model name is correct and you have internet access. Exiting.")
    exit()

# --- Data Loading and Preprocessing ---
try:
    with open(config.EXPLANATION_JSON, "r") as f:
        explanation_data_raw = json.load(f)
    print(f"Successfully loaded {len(explanation_data_raw)} items from {config.EXPLANATION_JSON}")
except FileNotFoundError:
    print(f"ERROR: JSON file not found at {config.EXPLANATION_JSON}. Please check the path. Exiting.")
    exit()
except json.JSONDecodeError:
    print(f"ERROR: JSON file at {config.EXPLANATION_JSON} is malformed. Please check the file content. Exiting.")
    exit()

# Convert to DataFrame for easier cleaning and inspection
df_raw = pd.DataFrame(explanation_data_raw)

# Basic Cleaning: Ensure 'file_name' and 'reason' columns exist and are not empty
if "file_name" not in df_raw.columns or "reason" not in df_raw.columns:
    print("ERROR: The JSON data must contain 'file_name' and 'reason' fields. Exiting.")
    exit()

df_cleaned = df_raw.dropna(subset=["file_name", "reason"])
df_cleaned = df_cleaned[df_cleaned["file_name"].str.strip() != ""]
# Ensure 'reason' is treated as a string before stripping, in case it's a list in the JSON
df_cleaned = df_cleaned[df_cleaned["reason"].astype(str).str.strip() != ""]


removed_count = len(df_raw) - len(df_cleaned)
if removed_count > 0:
    print(f"Removed {removed_count} rows with missing/empty 'file_name' or 'reason'.")

if len(df_cleaned) == 0:
    print("ERROR: No valid data remains after cleaning. Please check your JSON file content. Exiting.")
    exit()

# Create Positive Pairs (label=1)
positive_pairs = []
for _, row in df_cleaned.iterrows():
    positive_pairs.append({
        "file_name": row["file_name"],
        "reason": str(row["reason"]), # Ensure reason is string
        "label": 1
    })

# Create Negative Pairs (label=0)
all_reasons = df_cleaned["reason"].astype(str).unique().tolist()
negative_pairs = []

if len(all_reasons) <= 1 and len(positive_pairs) > 0 :
    print("WARNING: Only one unique reason available in the dataset. Negative pairs might not be very diverse or meaningful if they all use this single reason.")

for _, row in df_cleaned.iterrows():
    current_reason = str(row["reason"])
    possible_negative_reasons = [act for act in all_reasons if act != current_reason]

    if not possible_negative_reasons:
        # If only one unique reason exists, or if all reasons are the same as current_reason (shouldn't happen with unique list)
        # Fallback: pick any reason from the pool. If pool is empty, use a placeholder.
        negative_reason = random.choice(all_reasons) if all_reasons else "placeholder_reason_if_no_reasons_exist"
    else:
        negative_reason = random.choice(possible_negative_reasons)

    negative_pairs.append({
        "file_name": row["file_name"], # Same image, different reason
        "reason": negative_reason,
        "label": 0
    })

print(f"Created {len(positive_pairs)} positive pairs.")
print(f"Created {len(negative_pairs)} negative pairs.")

combined_data = positive_pairs + negative_pairs
if not combined_data:
    print("ERROR: No data available after creating positive and negative pairs. Exiting.")
    exit()

random.shuffle(combined_data) # Shuffle before splitting

# Stratified split to ensure balanced labels in train/val if possible
labels_for_split = [item['label'] for item in combined_data]
try:
    train_data, val_data = train_test_split(
        combined_data,
        test_size=config.VALIDATION_SPLIT,
        random_state=config.RANDOM_SEED,
        stratify=labels_for_split if len(set(labels_for_split)) > 1 else None # Stratify only if multiple classes
    )
except ValueError as e: # Handles cases where stratification is not possible (e.g., too few samples of a class)
    print(f"Warning: Could not stratify split (Reason: {e}). Performing random split.")
    train_data, val_data = train_test_split(
        combined_data,
        test_size=config.VALIDATION_SPLIT,
        random_state=config.RANDOM_SEED
    )

print(f"Total samples for training/validation: {len(combined_data)}")
print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")
if train_data: print(f"Sample training data item: {train_data[0]}")
if val_data: print(f"Sample validation data item: {val_data[0]}")


# --- Image Augmentations & Transformations ---
# Use the modern 'image_processor' attribute
image_processor = processor.image_processor
crop_config = image_processor.crop_size # e.g., {'height': 224, 'width': 224}

if isinstance(crop_config, dict) and 'height' in crop_config:
    image_transform_target_size = crop_config['height'] # Assuming square crop
elif isinstance(crop_config, int): # If crop_size is just an int (e.g. 224)
    image_transform_target_size = crop_config
else:
    print(f"Warning: Could not accurately determine target image size from image_processor.crop_size (value: {crop_config}). Defaulting to 224.")
    image_transform_target_size = 224

print(f"Image transform target size set to: {image_transform_target_size}x{image_transform_target_size}")

# CLIP's image mean and std for normalization
clip_image_mean = image_processor.image_mean
clip_image_std = image_processor.image_std

# Define transformations for training and validation
train_pil_to_tensor_transforms = T.Compose([
    T.RandomResizedCrop(image_transform_target_size, scale=(0.8, 1.0), ratio=(0.75, 1.33)), # More aggressive augmentation
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1), # Stronger color jitter
    T.RandomApply([T.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))], p=0.3), # Gaussian blur
    T.ToTensor(), # Converts to [0,1] and C,H,W
    T.Normalize(mean=clip_image_mean, std=clip_image_std),
])

val_pil_to_tensor_transforms = T.Compose([
    T.Resize(image_transform_target_size, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(image_transform_target_size),
    T.ToTensor(),
    T.Normalize(mean=clip_image_mean, std=clip_image_std),
])


# --- Custom Dataset ---
class CLIPSimilarityDataset(Dataset):
    def __init__(self, data, image_folder, tokenizer, transforms, max_text_length, placeholder_image_size):
        self.data = data
        self.image_folder = image_folder
        self.tokenizer = tokenizer
        self.transforms = transforms
        self.max_text_length = max_text_length
        self.placeholder_image_size = placeholder_image_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_path = os.path.join(self.image_folder, item["file_name"])

        try:
            image = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            # print(f"WARNING: Image not found {image_path}. Returning black image.") # Reduce verbosity for cleaner logs
            image = Image.new("RGB", (self.placeholder_image_size, self.placeholder_image_size), (0, 0, 0)) # Placeholder
        except Exception as e: # Catch all other PIL errors
            # print(f"WARNING: Error loading image {image_path}: {e}. Returning black image.") # Reduce verbosity
            image = Image.new("RGB", (self.placeholder_image_size, self.placeholder_image_size), (0, 0, 0)) # Placeholder

        # Apply PIL-to-Tensor transformations
        pixel_values = self.transforms(image)

        text = item["reason"]
        tokenized_text = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_text_length,
            return_tensors="pt"
        )

        input_ids = tokenized_text["input_ids"].squeeze(0) # Remove batch dim
        attention_mask = tokenized_text["attention_mask"].squeeze(0) # Remove batch dim
        label = torch.tensor(item["label"], dtype=torch.float) # Label is float for BCEWithLogitsLoss

        return {
            "pixel_values": pixel_values,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": label
        }

# Create Datasets
if not train_data:
    print("ERROR: Training data is empty. Cannot create DataLoader. Exiting.")
    exit()
train_dataset = CLIPSimilarityDataset(train_data, config.IMAGE_FOLDER, tokenizer, train_pil_to_tensor_transforms, config.MAX_LENGTH, image_transform_target_size)
val_dataset = CLIPSimilarityDataset(val_data, config.IMAGE_FOLDER, tokenizer, val_pil_to_tensor_transforms, config.MAX_LENGTH, image_transform_target_size)

# Create DataLoaders
# drop_last=True can help if the last batch is very small and causes issues, especially with contrastive loss (not an issue for BCE here but good practice)
train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train DataLoader created with {len(train_loader)} batches.")
if val_loader: print(f"Validation DataLoader created with {len(val_loader)} batches.")


# --- Optimizer, Scheduler, and Loss Function ---
optimizer = AdamW(clip_model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY, eps=1e-8)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=config.PLATEAU_FACTOR, patience=config.PLATEAU_PATIENCE, verbose=True)
bce_loss_fn = torch.nn.BCEWithLogitsLoss() # Loss function for binary classification

# --- Training and Validation ---
history = {
    "train_loss": [],
    "val_accuracy": [],
    "val_f1": [],
    "val_recall": [],
    "val_auroc": [],
    "learning_rate": []
}

best_val_metric_value = -1.0 # Initialize for 'max' mode metrics like AUROC/F1
epochs_no_improve = 0

# Initialize TorchMetrics objects once on the correct device
val_accuracy_metric_obj = Accuracy(task="binary", threshold=config.SIMILARITY_THRESHOLD).to(device)
val_f1_metric_obj = F1Score(task="binary", threshold=config.SIMILARITY_THRESHOLD).to(device)
val_recall_metric_obj = Recall(task="binary", threshold=config.SIMILARITY_THRESHOLD).to(device)
val_auroc_metric_obj = AUROC(task="binary").to(device) # AUROC typically takes logits or raw scores

print("\n--- Starting Training with BCE Loss ---")
for epoch in range(config.NUM_EPOCHS):
    clip_model.train() # Set model to training mode
    total_train_loss = 0.0

    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS} [TRAIN]", leave=False)
    for batch in train_pbar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        # Labels (0/1) for the pairs, ensure shape (batch_size, 1) for BCEWithLogitsLoss
        actual_pair_labels = batch["labels"].to(device).unsqueeze(1)

        optimizer.zero_grad() # Clear previous gradients

        # Get image and text features (embeddings)
        image_features = clip_model.get_image_features(pixel_values=pixel_values)
        text_features = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)

        # Normalize features (important for stable dot product/cosine similarity)
        image_features_norm = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features_norm = text_features / text_features.norm(dim=-1, keepdim=True)

        # Calculate dot product and scale by CLIP's learned logit_scale.
        # This gives us the "logits" for our binary classification task.
        similarity_logits = (image_features_norm * text_features_norm).sum(dim=1) * clip_model.logit_scale.exp()
        similarity_logits = similarity_logits.unsqueeze(1) # Ensure shape (batch_size, 1)

        # Calculate loss using BCEWithLogitsLoss
        loss = bce_loss_fn(similarity_logits, actual_pair_labels)

        loss.backward() # Backpropagate the loss
        torch.nn.utils.clip_grad_norm_(clip_model.parameters(), config.GRADIENT_CLIP_VAL) # Gradient clipping
        optimizer.step() # Update model parameters

        total_train_loss += loss.item()
        train_pbar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    history["train_loss"].append(avg_train_loss)
    history["learning_rate"].append(optimizer.param_groups[0]['lr'])
    print(f"Epoch {epoch+1} Avg Train Loss: {avg_train_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.2e}")

    # --- Validation ---
    if not val_loader: # Skip validation if no val_loader
        if not isinstance(scheduler, ReduceLROnPlateau): scheduler.step() # Step schedulers that don't need metrics
        continue

    clip_model.eval() # Set model to evaluation mode
    all_val_labels_list = []
    all_val_similarity_logits_list = [] # Store raw logits for AUROC

    val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS} [VALIDATE]", leave=False)
    with torch.no_grad(): # Disable gradient calculations for validation
        for batch in val_pbar:
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device) # Ground truth labels

            # Get embeddings
            image_embeds = clip_model.get_image_features(pixel_values=pixel_values)
            text_embeds = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
            image_embeds_norm = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
            text_embeds_norm = text_embeds / text_embeds.norm(dim=-1, keepdim=True)

            # Calculate scaled similarity scores (logits) for evaluation
            similarity_logits_for_eval = (image_embeds_norm * text_embeds_norm).sum(dim=1) * clip_model.logit_scale.exp()

            all_val_labels_list.append(labels)
            all_val_similarity_logits_list.append(similarity_logits_for_eval)

    # Concatenate results from all validation batches
    all_val_labels_tensor = torch.cat(all_val_labels_list)
    all_val_similarity_logits_tensor = torch.cat(all_val_similarity_logits_list)

    # Convert logits to probabilities for threshold-based metrics (Accuracy, F1, Recall)
    all_val_probs_tensor = torch.sigmoid(all_val_similarity_logits_tensor)

    # Compute metrics using TorchMetrics objects
    current_val_accuracy = val_accuracy_metric_obj(all_val_probs_tensor, all_val_labels_tensor.int())
    current_val_f1 = val_f1_metric_obj(all_val_probs_tensor, all_val_labels_tensor.int())
    current_val_recall = val_recall_metric_obj(all_val_probs_tensor, all_val_labels_tensor.int())
    current_val_auroc = val_auroc_metric_obj(all_val_similarity_logits_tensor, all_val_labels_tensor.int()) # AUROC uses logits

    # Store metrics in history
    history["val_accuracy"].append(current_val_accuracy.item())
    history["val_f1"].append(current_val_f1.item())
    history["val_recall"].append(current_val_recall.item())
    history["val_auroc"].append(current_val_auroc.item())

    print(f"Epoch {epoch+1} VAL - Acc: {current_val_accuracy:.4f} (Th: {config.SIMILARITY_THRESHOLD}), F1: {current_val_f1:.4f}, Recall: {current_val_recall:.4f}, AUROC: {current_val_auroc:.4f}")
    print(f"Epoch {epoch+1} VAL - Min/Max Similarity LOGITS: {all_val_similarity_logits_tensor.min().item():.4f} / {all_val_similarity_logits_tensor.max().item():.4f}")

    # Reset metrics for the next epoch (TorchMetrics objects accumulate state unless reset)
    val_accuracy_metric_obj.reset()
    val_f1_metric_obj.reset()
    val_recall_metric_obj.reset()
    val_auroc_metric_obj.reset()

    # --- Scheduler Step & Early Stopping ---
    metric_to_monitor_val = current_val_auroc # Defaulting to AUROC
    if config.METRIC_FOR_EARLY_STOPPING == "val_f1":
        metric_to_monitor_val = current_val_f1
    elif config.METRIC_FOR_EARLY_STOPPING == "val_accuracy":
        metric_to_monitor_val = current_val_accuracy

    if isinstance(scheduler, ReduceLROnPlateau):
        scheduler.step(metric_to_monitor_val)
    elif isinstance(scheduler, CosineAnnealingLR): # Or other schedulers not needing a metric
        scheduler.step()

    if metric_to_monitor_val.item() > best_val_metric_value: # Ensure comparing scalar values
        print(f"Validation metric ({config.METRIC_FOR_EARLY_STOPPING}) improved from {best_val_metric_value:.4f} to {metric_to_monitor_val.item():.4f}. Saving model.")
        best_val_metric_value = metric_to_monitor_val.item()
        torch.save(clip_model.state_dict(), config.MODEL_SAVE_PATH)
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"Validation metric ({config.METRIC_FOR_EARLY_STOPPING}) did not improve for {epochs_no_improve} epoch(s). Best was {best_val_metric_value:.4f}.")

    if epochs_no_improve >= config.EARLY_STOPPING_PATIENCE:
        print(f"Early stopping triggered after {config.EARLY_STOPPING_PATIENCE} epochs without improvement on {config.METRIC_FOR_EARLY_STOPPING} at epoch {epoch+1}.")
        break

print("\n--- Training Finished ---")

# --- Plot Training History ---
def plot_history(history_dict, config_obj):
    epochs_ran = len(history_dict.get('train_loss', []))
    if epochs_ran == 0:
        print("No training data to plot.")
        return

    plt.figure(figsize=(18, 6))

    # Plot Training Loss
    plt.subplot(1, 3, 1)
    plt.plot(range(1, epochs_ran + 1), history_dict['train_loss'], label='Train Loss', marker='o')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot Validation Metrics
    plt.subplot(1, 3, 2)
    if history_dict.get('val_auroc'):
      plt.plot(range(1, epochs_ran + 1), history_dict['val_auroc'], label='Val AUROC', marker='o', color='red')
    if history_dict.get('val_f1'):
      plt.plot(range(1, epochs_ran + 1), history_dict['val_f1'], label=f'Val F1 (Th={config_obj.SIMILARITY_THRESHOLD})', marker='o', color='orange')
    if history_dict.get('val_accuracy'):
      plt.plot(range(1, epochs_ran + 1), history_dict['val_accuracy'], label=f'Val Acc (Th={config_obj.SIMILARITY_THRESHOLD})', marker='o', color='green')
    plt.title('Validation Metrics over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.grid(True)

    # Plot Learning Rate
    plt.subplot(1, 3, 3)
    if history_dict.get('learning_rate'):
      plt.plot(range(1, epochs_ran + 1), history_dict['learning_rate'], label='Learning Rate', marker='o', color='purple')
    plt.title('Learning Rate over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('LR')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(config_obj.MODEL_SAVE_DIR, "training_history_bce.png"))
    plt.show()

plot_history(history, config)


# --- Final Evaluation on Validation Set (using the best model) ---
print("\n--- Final Evaluation on Validation Set using Best Model ---")
if os.path.exists(config.MODEL_SAVE_PATH):
    try:
        clip_model.load_state_dict(torch.load(config.MODEL_SAVE_PATH, map_location=device))
        print(f"Successfully loaded best model from {config.MODEL_SAVE_PATH}")
    except Exception as e:
        print(f"Error loading best model: {e}. Evaluating with the last model state.")
else:
    print(f"Best model path {config.MODEL_SAVE_PATH} not found. Evaluating with the last model state (if any training happened).")

clip_model.eval() # Ensure model is in evaluation mode
final_all_labels_list = []
final_all_similarity_logits_list = [] # Store raw logits

final_eval_pbar = tqdm(val_loader, desc="Final Evaluation", leave=False)
with torch.no_grad():
    for batch in final_eval_pbar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].cpu() # Collect labels on CPU

        # Get embeddings and calculate logits
        image_embeds = clip_model.get_image_features(pixel_values=pixel_values)
        text_embeds = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        image_embeds_norm = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
        text_embeds_norm = text_embeds / text_embeds.norm(dim=-1, keepdim=True)
        similarity_logits_for_eval = ((image_embeds_norm * text_embeds_norm).sum(dim=1) * clip_model.logit_scale.exp()).cpu() # Collect logits on CPU

        final_all_labels_list.append(labels)
        final_all_similarity_logits_list.append(similarity_logits_for_eval)

# Concatenate all results
final_labels_tensor = torch.cat(final_all_labels_list)
final_similarity_logits_tensor = torch.cat(final_all_similarity_logits_list)

# Convert logits to probabilities for threshold-based metrics
final_probs_tensor = torch.sigmoid(final_similarity_logits_tensor)

# Re-initialize metrics for final evaluation for clarity
final_accuracy_metric = Accuracy(task="binary", threshold=config.SIMILARITY_THRESHOLD)
final_f1_metric = F1Score(task="binary", threshold=config.SIMILARITY_THRESHOLD)
final_recall_metric = Recall(task="binary", threshold=config.SIMILARITY_THRESHOLD)
final_auroc_metric = AUROC(task="binary") # Takes logits or probabilities

# Calculate final metrics
final_accuracy = final_accuracy_metric(final_probs_tensor, final_labels_tensor.int())
final_f1 = final_f1_metric(final_probs_tensor, final_labels_tensor.int())
final_recall = final_recall_metric(final_probs_tensor, final_labels_tensor.int())
final_auroc = final_auroc_metric(final_similarity_logits_tensor, final_labels_tensor.int()) # AUROC uses logits

print(f"\nFinal Validation Metrics (Threshold for Acc/F1/Recall: {config.SIMILARITY_THRESHOLD}):")
print(f"  Accuracy: {final_accuracy:.4f}")
print(f"  F1 Score: {final_f1:.4f}")
print(f"  Recall:   {final_recall:.4f}")
print(f"  ROC AUC:  {final_auroc:.4f}")


# --- Confusion Matrix ---
# Predictions for confusion matrix based on the chosen threshold
final_preds_binary = (final_probs_tensor >= config.SIMILARITY_THRESHOLD).int()

# Using TorchMetrics ConfusionMatrix
cm_metric = ConfusionMatrix(task="binary", num_classes=2) # Threshold is handled by inputting binary predictions
cm_tensor = cm_metric(final_preds_binary, final_labels_tensor.int())
cm = cm_tensor.cpu().numpy() # Convert to NumPy array for Seaborn

plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred Mismatch (0)", "Pred Match (1)"],
            yticklabels=["Actual Mismatch (0)", "Actual Match (1)"])
plt.title(f"Confusion Matrix (Threshold: {config.SIMILARITY_THRESHOLD})")
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")
plt.savefig(os.path.join(config.MODEL_SAVE_DIR, "confusion_matrix_bce.png"))
plt.show()

# --- ROC Curve ---
# Use raw logits for a smoother ROC curve
y_true_np = final_labels_tensor.numpy().astype(int)
y_scores_np = final_similarity_logits_tensor.numpy() # Using logits for roc_curve

fpr, tpr, thresholds_roc = sk_roc_curve(y_true_np, y_scores_np)

plt.figure(figsize=(7, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {final_auroc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') # Diagonal random guess line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.savefig(os.path.join(config.MODEL_SAVE_DIR, "roc_curve_bce.png"))
plt.show()

print(f"\nAll artifacts (model, plots) saved in: {config.MODEL_SAVE_DIR}")
print("--- Script Finished ---")

#Train1400, val200, test400


In [ ]:
# Standard library imports
import os
import json
import random
import numpy as np
import pandas as pd # For easier data handling

# PyTorch and related imports
import torch
import torch.nn.functional as F # For functional operations if needed
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

# Hugging Face Transformers imports
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizerFast

# Scikit-learn imports for utilities and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve as sk_roc_curve # For plotting ROC

# Other useful libraries
from tqdm.auto import tqdm # For progress bars
from PIL import Image, ImageFile # For image manipulation
import matplotlib.pyplot as plt # For plotting
import seaborn as sns # For enhanced plotting (like heatmaps)
import torchvision.transforms as T # For image transformations

# TorchMetrics for efficient and correct metric calculation
from torchmetrics import Accuracy, F1Score, Recall, AUROC, ConfusionMatrix

# Configuration to allow loading of potentially truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --- Configuration Class ---
class Config:
    # Data paths
    IMAGE_FOLDER = "/content/drive/MyDrive/RL_final_project/bdd_for_clip/data"  # CHANGE THIS TO YOUR IMAGE FOLDER PATH
    EXPLANATION_JSON = "/content/drive/MyDrive/RL_final_project/bdd_all_1a1r.json" # CHANGE THIS TO YOUR JSON PATH

    # Model configuration
    MODEL_NAME = "openai/clip-vit-base-patch32" # Pre-trained CLIP model

    # Training hyperparameters
    BATCH_SIZE = 32  # Adjust based on GPU memory
    NUM_EPOCHS = 30  # Increased epochs, use with early stopping
    LEARNING_RATE = 2e-5  # A common starting point for fine-tuning transformers with BCE
    WEIGHT_DECAY = 0.01 # Weight decay for regularization
    GRADIENT_CLIP_VAL = 1.0 # For gradient clipping to prevent exploding gradients

    # Scheduler parameters (using ReduceLROnPlateau as an example)
    PLATEAU_FACTOR = 0.2  # Factor by which the learning rate will be reduced
    PLATEAU_PATIENCE = 3  # Number of epochs with no improvement after which learning rate will be reduced

    # Data splitting and processing
    VALIDATION_SPLIT = 0.2  # Proportion of data to use for validation (NO LONGER USED FOR MAIN SPLIT)
    RANDOM_SEED = 42  # For reproducibility
    MAX_LENGTH = 77  # Max sequence length for CLIP tokenizer

    # Model saving
    MODEL_SAVE_DIR = "/content/drive/MyDrive/RL_final_project/clip_finetuned_models_reason/" # Directory to save models
    BEST_MODEL_NAME = "best_clip_judgement_model_reason.pth" # Name for the best model file

    # Early stopping
    EARLY_STOPPING_PATIENCE = 7  # Number of epochs to wait for improvement before stopping
    METRIC_FOR_EARLY_STOPPING = "val_auroc"  # Metric to monitor for early stopping (e.g., "val_f1", "val_accuracy")

    # Threshold for converting probabilities (after sigmoid) to binary predictions (0 or 1)
    SIMILARITY_THRESHOLD = 0.5  # Standard threshold for probabilities from sigmoid

# Instantiate configuration
config = Config()

# Create model save directory if it doesn't exist
os.makedirs(config.MODEL_SAVE_DIR, exist_ok=True)
config.MODEL_SAVE_PATH = os.path.join(config.MODEL_SAVE_DIR, config.BEST_MODEL_NAME)

# --- Utility Functions ---
def set_seed(seed):
    """Sets the seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        # The following two lines can ensure full reproducibility but might slow down training
        # torch.backends.cudnn.deterministic = True
        # torch.backends.cudnn.benchmark = False

set_seed(config.RANDOM_SEED)

# --- Hugging Face Login (Optional) ---
# (Original code for Hugging Face login can be kept here if needed)
# hf_token = userdata.get("hf_token")
# if hf_token:
# try:
# login(token=hf_token)
# print("Successfully logged into Hugging Face.")
#     except Exception as e:
# print(f"Hugging Face login failed: {e}. Proceeding without login.")
# else:
# print("Hugging Face token not found in Colab secrets. Proceeding without login.")

# --- Device Configuration ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Load CLIP Model, Processor, and Tokenizer ---
try:
    clip_model = CLIPModel.from_pretrained(config.MODEL_NAME).to(device)
    processor = CLIPProcessor.from_pretrained(config.MODEL_NAME)
    tokenizer = CLIPTokenizerFast.from_pretrained(config.MODEL_NAME)
    print(f"CLIP model '{config.MODEL_NAME}' and processor loaded successfully.")
except Exception as e:
    print(f"Error loading CLIP model or processor: {e}. Please ensure the model name is correct and you have internet access. Exiting.")
    exit()

# --- Data Loading and Preprocessing ---
try:
    with open(config.EXPLANATION_JSON, "r") as f:
        explanation_data_raw = json.load(f)
    print(f"Successfully loaded {len(explanation_data_raw)} items from {config.EXPLANATION_JSON}")
except FileNotFoundError:
    print(f"ERROR: JSON file not found at {config.EXPLANATION_JSON}. Please check the path. Exiting.")
    exit()
except json.JSONDecodeError:
    print(f"ERROR: JSON file at {config.EXPLANATION_JSON} is malformed. Please check the file content. Exiting.")
    exit()

df_raw = pd.DataFrame(explanation_data_raw)

if "file_name" not in df_raw.columns or "reason" not in df_raw.columns:
    print("ERROR: The JSON data must contain 'file_name' and 'reason' fields. Exiting.")
    exit()

df_cleaned = df_raw.dropna(subset=["file_name", "reason"])
df_cleaned = df_cleaned[df_cleaned["file_name"].str.strip() != ""]
df_cleaned = df_cleaned[df_cleaned["reason"].astype(str).str.strip() != ""]

removed_count = len(df_raw) - len(df_cleaned)
if removed_count > 0:
    print(f"Removed {removed_count} rows with missing/empty 'file_name' or 'reason'.")

if len(df_cleaned) == 0:
    print("ERROR: No valid data remains after cleaning. Please check your JSON file content. Exiting.")
    exit()

positive_pairs = []
for _, row in df_cleaned.iterrows():
    positive_pairs.append({
        "file_name": row["file_name"],
        "reason": str(row["reason"]),
        "label": 1
    })

all_reasons = df_cleaned["reason"].astype(str).unique().tolist()
negative_pairs = []

if len(all_reasons) <= 1 and len(positive_pairs) > 0 :
    print("WARNING: Only one unique reason available in the dataset. Negative pairs might not be very diverse or meaningful.")

for _, row in df_cleaned.iterrows():
    current_reason = str(row["reason"])
    possible_negative_reasons = [act for act in all_reasons if act != current_reason]
    negative_reason = random.choice(possible_negative_reasons) if possible_negative_reasons else (random.choice(all_reasons) if all_reasons else "placeholder_reason")
    negative_pairs.append({
        "file_name": row["file_name"],
        "reason": negative_reason,
        "label": 0
    })

print(f"Created {len(positive_pairs)} positive pairs.")
print(f"Created {len(negative_pairs)} negative pairs.")

combined_data = positive_pairs + negative_pairs
if not combined_data:
    print("ERROR: No data available after creating positive and negative pairs. Exiting.")
    exit()

random.shuffle(combined_data)

# --- MODIFIED DATA SPLITTING ---
TRAIN_SIZE = 1400
VAL_SIZE = 200
TEST_SIZE = 400
TOTAL_REQUESTED_SAMPLES = TRAIN_SIZE + VAL_SIZE + TEST_SIZE

print(f"Total combined samples before split: {len(combined_data)}")
print(f"Requested split: Train={TRAIN_SIZE}, Val={VAL_SIZE}, Test={TEST_SIZE} (Total: {TOTAL_REQUESTED_SAMPLES})")

if len(combined_data) < TOTAL_REQUESTED_SAMPLES:
    print(f"ERROR: Not enough data ({len(combined_data)}) for the requested split. Need {TOTAL_REQUESTED_SAMPLES}. Exiting.")
    exit()

labels_for_split = [item['label'] for item in combined_data]
stratify_array = np.array(labels_for_split) if len(set(labels_for_split)) > 1 else None

try:
    # Split into train and (val + test)
    # We use indices to manage data for multi-stage splitting with stratification
    indices = np.arange(len(combined_data))

    train_indices, remaining_indices = train_test_split(
        indices,
        train_size=TRAIN_SIZE,
        test_size=(VAL_SIZE + TEST_SIZE),
        random_state=config.RANDOM_SEED,
        stratify=stratify_array if stratify_array is not None else None
    )

    # Prepare stratification for the second split (val/test)
    if stratify_array is not None:
        stratify_for_remaining = stratify_array[remaining_indices]
    else:
        stratify_for_remaining = None

    # Split remaining into val and test
    val_indices_temp, test_indices_temp = train_test_split(
        remaining_indices, # Split the actual indices from the previous step
        train_size=VAL_SIZE,
        test_size=TEST_SIZE,
        random_state=config.RANDOM_SEED, # Using the same seed for reproducibility of this second split
        stratify=stratify_for_remaining if stratify_for_remaining is not None else None
    )

    train_data = [combined_data[i] for i in train_indices]
    val_data = [combined_data[i] for i in val_indices_temp]
    test_data = [combined_data[i] for i in test_indices_temp]

except ValueError as e:
    print(f"ERROR: Could not perform stratified split with fixed sizes (Reason: {e}). "
          f"This often means a class has too few samples for the requested split sizes/proportions. "
          f"Please check your data distribution or adjust split sizes. Exiting.")
    exit()

print(f"Actual training samples: {len(train_data)}")
print(f"Actual validation samples: {len(val_data)}")
print(f"Actual test samples: {len(test_data)}")

if len(train_data) != TRAIN_SIZE or len(val_data) != VAL_SIZE or len(test_data) != TEST_SIZE:
    print("WARNING: Actual split sizes differ from requested. This may occur if stratification constraints conflict with exact counts.")
    # Potentially exit or handle as per requirements if exact counts are critical despite stratification issues.

if train_data: print(f"Sample training data item: {train_data[0]}")
if val_data: print(f"Sample validation data item: {val_data[0]}")
if test_data: print(f"Sample test data item: {test_data[0]}")
# --- END OF MODIFIED DATA SPLITTING ---

# --- Image Augmentations & Transformations ---
image_processor = processor.image_processor
crop_config = image_processor.crop_size

if isinstance(crop_config, dict) and 'height' in crop_config:
    image_transform_target_size = crop_config['height']
elif isinstance(crop_config, int):
    image_transform_target_size = crop_config
else:
    print(f"Warning: Could not accurately determine target image size from image_processor.crop_size (value: {crop_config}). Defaulting to 224.")
    image_transform_target_size = 224

print(f"Image transform target size set to: {image_transform_target_size}x{image_transform_target_size}")

clip_image_mean = image_processor.image_mean
clip_image_std = image_processor.image_std

train_pil_to_tensor_transforms = T.Compose([
    T.RandomResizedCrop(image_transform_target_size, scale=(0.8, 1.0), ratio=(0.75, 1.33)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    T.RandomApply([T.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))], p=0.3),
    T.ToTensor(),
    T.Normalize(mean=clip_image_mean, std=clip_image_std),
])

val_pil_to_tensor_transforms = T.Compose([
    T.Resize(image_transform_target_size, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(image_transform_target_size),
    T.ToTensor(),
    T.Normalize(mean=clip_image_mean, std=clip_image_std),
])

# --- Custom Dataset ---
class CLIPSimilarityDataset(Dataset):
    def __init__(self, data, image_folder, tokenizer, transforms, max_text_length, placeholder_image_size):
        self.data = data
        self.image_folder = image_folder
        self.tokenizer = tokenizer
        self.transforms = transforms
        self.max_text_length = max_text_length
        self.placeholder_image_size = placeholder_image_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_path = os.path.join(self.image_folder, item["file_name"])
        try:
            image = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            image = Image.new("RGB", (self.placeholder_image_size, self.placeholder_image_size), (0, 0, 0))
        except Exception as e:
            image = Image.new("RGB", (self.placeholder_image_size, self.placeholder_image_size), (0, 0, 0))

        pixel_values = self.transforms(image)
        text = item["reason"]
        tokenized_text = self.tokenizer(
            text, padding='max_length', truncation=True, max_length=self.max_text_length, return_tensors="pt"
        )
        input_ids = tokenized_text["input_ids"].squeeze(0)
        attention_mask = tokenized_text["attention_mask"].squeeze(0)
        label = torch.tensor(item["label"], dtype=torch.float)
        return {"pixel_values": pixel_values, "input_ids": input_ids, "attention_mask": attention_mask, "labels": label}

if not train_data:
    print("ERROR: Training data is empty. Cannot create DataLoader. Exiting.")
    exit()
train_dataset = CLIPSimilarityDataset(train_data, config.IMAGE_FOLDER, tokenizer, train_pil_to_tensor_transforms, config.MAX_LENGTH, image_transform_target_size)
val_dataset = CLIPSimilarityDataset(val_data, config.IMAGE_FOLDER, tokenizer, val_pil_to_tensor_transforms, config.MAX_LENGTH, image_transform_target_size)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train DataLoader created with {len(train_loader)} batches.")
if val_loader: print(f"Validation DataLoader created with {len(val_loader)} batches.")

# Create Test DataLoader
if not test_data:
    print("WARNING: Test data is empty. Test DataLoader will not be created.")
    test_loader = None
else:
    test_dataset = CLIPSimilarityDataset(test_data, config.IMAGE_FOLDER, tokenizer, val_pil_to_tensor_transforms, config.MAX_LENGTH, image_transform_target_size)
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    print(f"Test DataLoader created with {len(test_loader)} batches.")

# --- Optimizer, Scheduler, and Loss Function ---
optimizer = AdamW(clip_model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY, eps=1e-8)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=config.PLATEAU_FACTOR, patience=config.PLATEAU_PATIENCE, verbose=True)
bce_loss_fn = torch.nn.BCEWithLogitsLoss()

# --- Training and Validation ---
history = {"train_loss": [], "val_accuracy": [], "val_f1": [], "val_recall": [], "val_auroc": [], "learning_rate": []}
best_val_metric_value = -1.0
epochs_no_improve = 0

val_accuracy_metric_obj = Accuracy(task="binary", threshold=config.SIMILARITY_THRESHOLD).to(device)
val_f1_metric_obj = F1Score(task="binary", threshold=config.SIMILARITY_THRESHOLD).to(device)
val_recall_metric_obj = Recall(task="binary", threshold=config.SIMILARITY_THRESHOLD).to(device)
val_auroc_metric_obj = AUROC(task="binary").to(device)

print("\n--- Starting Training with BCE Loss ---")
for epoch in range(config.NUM_EPOCHS):
    clip_model.train()
    total_train_loss = 0.0
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS} [TRAIN]", leave=False)
    for batch in train_pbar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        actual_pair_labels = batch["labels"].to(device).unsqueeze(1)
        optimizer.zero_grad()
        image_features = clip_model.get_image_features(pixel_values=pixel_values)
        text_features = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        image_features_norm = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features_norm = text_features / text_features.norm(dim=-1, keepdim=True)
        similarity_logits = (image_features_norm * text_features_norm).sum(dim=1) * clip_model.logit_scale.exp()
        similarity_logits = similarity_logits.unsqueeze(1)
        loss = bce_loss_fn(similarity_logits, actual_pair_labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(clip_model.parameters(), config.GRADIENT_CLIP_VAL)
        optimizer.step()
        total_train_loss += loss.item()
        train_pbar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    history["train_loss"].append(avg_train_loss)
    history["learning_rate"].append(optimizer.param_groups[0]['lr'])
    print(f"Epoch {epoch+1} Avg Train Loss: {avg_train_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.2e}")

    if not val_loader:
        if not isinstance(scheduler, ReduceLROnPlateau): scheduler.step()
        continue

    clip_model.eval()
    all_val_labels_list = []
    all_val_similarity_logits_list = []
    val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS} [VALIDATE]", leave=False)
    with torch.no_grad():
        for batch in val_pbar:
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            image_embeds = clip_model.get_image_features(pixel_values=pixel_values)
            text_embeds = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
            image_embeds_norm = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
            text_embeds_norm = text_embeds / text_embeds.norm(dim=-1, keepdim=True) # Corrected from text_features
            similarity_logits_for_eval = (image_embeds_norm * text_embeds_norm).sum(dim=1) * clip_model.logit_scale.exp()
            all_val_labels_list.append(labels)
            all_val_similarity_logits_list.append(similarity_logits_for_eval)

    all_val_labels_tensor = torch.cat(all_val_labels_list)
    all_val_similarity_logits_tensor = torch.cat(all_val_similarity_logits_list)
    all_val_probs_tensor = torch.sigmoid(all_val_similarity_logits_tensor)

    current_val_accuracy = val_accuracy_metric_obj(all_val_probs_tensor, all_val_labels_tensor.int())
    current_val_f1 = val_f1_metric_obj(all_val_probs_tensor, all_val_labels_tensor.int())
    current_val_recall = val_recall_metric_obj(all_val_probs_tensor, all_val_labels_tensor.int())
    current_val_auroc = val_auroc_metric_obj(all_val_similarity_logits_tensor, all_val_labels_tensor.int())

    history["val_accuracy"].append(current_val_accuracy.item())
    history["val_f1"].append(current_val_f1.item())
    history["val_recall"].append(current_val_recall.item())
    history["val_auroc"].append(current_val_auroc.item())
    print(f"Epoch {epoch+1} VAL - Acc: {current_val_accuracy:.4f} (Th: {config.SIMILARITY_THRESHOLD}), F1: {current_val_f1:.4f}, Recall: {current_val_recall:.4f}, AUROC: {current_val_auroc:.4f}")

    val_accuracy_metric_obj.reset()
    val_f1_metric_obj.reset()
    val_recall_metric_obj.reset()
    val_auroc_metric_obj.reset()

    metric_to_monitor_val = current_val_auroc
    if config.METRIC_FOR_EARLY_STOPPING == "val_f1": metric_to_monitor_val = current_val_f1
    elif config.METRIC_FOR_EARLY_STOPPING == "val_accuracy": metric_to_monitor_val = current_val_accuracy

    if isinstance(scheduler, ReduceLROnPlateau): scheduler.step(metric_to_monitor_val)
    elif isinstance(scheduler, CosineAnnealingLR): scheduler.step()

    if metric_to_monitor_val.item() > best_val_metric_value:
        print(f"Validation metric ({config.METRIC_FOR_EARLY_STOPPING}) improved from {best_val_metric_value:.4f} to {metric_to_monitor_val.item():.4f}. Saving model.")
        best_val_metric_value = metric_to_monitor_val.item()
        torch.save(clip_model.state_dict(), config.MODEL_SAVE_PATH)
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"Validation metric ({config.METRIC_FOR_EARLY_STOPPING}) did not improve for {epochs_no_improve} epoch(s). Best was {best_val_metric_value:.4f}.")
    if epochs_no_improve >= config.EARLY_STOPPING_PATIENCE:
        print(f"Early stopping triggered after {config.EARLY_STOPPING_PATIENCE} epochs without improvement on {config.METRIC_FOR_EARLY_STOPPING} at epoch {epoch+1}.")
        break
print("\n--- Training Finished ---")

# --- Plot Training History ---
def plot_history(history_dict, config_obj):
    epochs_ran = len(history_dict.get('train_loss', []))
    if epochs_ran == 0: print("No training data to plot."); return
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.plot(range(1, epochs_ran + 1), history_dict['train_loss'], label='Train Loss', marker='o')
    plt.title('Loss over Epochs'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.subplot(1, 3, 2)
    if history_dict.get('val_auroc'): plt.plot(range(1, epochs_ran + 1), history_dict['val_auroc'], label='Val AUROC', marker='o', color='red')
    if history_dict.get('val_f1'): plt.plot(range(1, epochs_ran + 1), history_dict['val_f1'], label=f'Val F1 (Th={config_obj.SIMILARITY_THRESHOLD})', marker='o', color='orange')
    if history_dict.get('val_accuracy'): plt.plot(range(1, epochs_ran + 1), history_dict['val_accuracy'], label=f'Val Acc (Th={config_obj.SIMILARITY_THRESHOLD})', marker='o', color='green')
    plt.title('Validation Metrics over Epochs'); plt.xlabel('Epoch'); plt.ylabel('Metric Value'); plt.legend(); plt.grid(True)
    plt.subplot(1, 3, 3)
    if history_dict.get('learning_rate'): plt.plot(range(1, epochs_ran + 1), history_dict['learning_rate'], label='Learning Rate', marker='o', color='purple')
    plt.title('Learning Rate over Epochs'); plt.xlabel('Epoch'); plt.ylabel('LR'); plt.legend(); plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(config_obj.MODEL_SAVE_DIR, "training_history_bce.png")); plt.show()
plot_history(history, config)

# --- Final Evaluation on Validation Set (using the best model) ---
print("\n--- Final Evaluation on Validation Set using Best Model ---")
if os.path.exists(config.MODEL_SAVE_PATH):
    try:
        clip_model.load_state_dict(torch.load(config.MODEL_SAVE_PATH, map_location=device))
        print(f"Successfully loaded best model from {config.MODEL_SAVE_PATH}")
    except Exception as e:
        print(f"Error loading best model: {e}. Evaluating with the last model state.")
else:
    print(f"Best model path {config.MODEL_SAVE_PATH} not found. Evaluating with the last model state.")

clip_model.eval()
final_all_labels_list = []
final_all_similarity_logits_list = []
final_eval_pbar = tqdm(val_loader, desc="Final Validation Evaluation", leave=False)
with torch.no_grad():
    for batch in final_eval_pbar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].cpu()
        image_embeds = clip_model.get_image_features(pixel_values=pixel_values)
        text_embeds = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        image_embeds_norm = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
        text_embeds_norm = text_embeds / text_embeds.norm(dim=-1, keepdim=True) # CORRECTED from text_features
        similarity_logits_for_eval = ((image_embeds_norm * text_embeds_norm).sum(dim=1) * clip_model.logit_scale.exp()).cpu()
        final_all_labels_list.append(labels)
        final_all_similarity_logits_list.append(similarity_logits_for_eval)

final_labels_tensor = torch.cat(final_all_labels_list)
final_similarity_logits_tensor = torch.cat(final_all_similarity_logits_list)
final_probs_tensor = torch.sigmoid(final_similarity_logits_tensor)

final_accuracy_metric = Accuracy(task="binary", threshold=config.SIMILARITY_THRESHOLD)
final_f1_metric = F1Score(task="binary", threshold=config.SIMILARITY_THRESHOLD)
final_recall_metric = Recall(task="binary", threshold=config.SIMILARITY_THRESHOLD)
final_auroc_metric = AUROC(task="binary")

final_accuracy = final_accuracy_metric(final_probs_tensor, final_labels_tensor.int())
final_f1 = final_f1_metric(final_probs_tensor, final_labels_tensor.int())
final_recall = final_recall_metric(final_probs_tensor, final_labels_tensor.int())
final_auroc = final_auroc_metric(final_similarity_logits_tensor, final_labels_tensor.int())

print(f"\nFinal Validation Metrics (Threshold: {config.SIMILARITY_THRESHOLD}):")
print(f"  Accuracy: {final_accuracy:.4f}, F1 Score: {final_f1:.4f}, Recall: {final_recall:.4f}, ROC AUC: {final_auroc:.4f}")

final_preds_binary = (final_probs_tensor >= config.SIMILARITY_THRESHOLD).int()
cm_metric = ConfusionMatrix(task="binary", num_classes=2)
cm_tensor = cm_metric(final_preds_binary, final_labels_tensor.int())
cm = cm_tensor.cpu().numpy()
plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Pred Mismatch (0)", "Pred Match (1)"], yticklabels=["Actual Mismatch (0)", "Actual Match (1)"])
plt.title(f"Validation Confusion Matrix (Th: {config.SIMILARITY_THRESHOLD})"); plt.xlabel("Predicted Label"); plt.ylabel("Actual Label")
plt.savefig(os.path.join(config.MODEL_SAVE_DIR, "confusion_matrix_validation_bce.png")); plt.show()

y_true_np = final_labels_tensor.numpy().astype(int)
y_scores_np = final_similarity_logits_tensor.numpy()
fpr, tpr, _ = sk_roc_curve(y_true_np, y_scores_np)
plt.figure(figsize=(7, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {final_auroc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--'); plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('Validation ROC Curve')
plt.legend(loc="lower right"); plt.grid(True)
plt.savefig(os.path.join(config.MODEL_SAVE_DIR, "roc_curve_validation_bce.png")); plt.show()


# --- Final Evaluation on Test Set (using the best model) ---
if test_loader:
    print("\n--- Final Evaluation on Test Set using Best Model ---")
    # Model should already be loaded or use the last state if loading failed previously.
    # Re-asserting best model load for clarity, though it might be redundant if no errors occurred.
    if os.path.exists(config.MODEL_SAVE_PATH):
        try:
            clip_model.load_state_dict(torch.load(config.MODEL_SAVE_PATH, map_location=device))
            print(f"Successfully re-loaded best model from {config.MODEL_SAVE_PATH} for test evaluation.")
        except Exception as e:
            print(f"Error re-loading best model for test set: {e}. Evaluating with current model state.")
    else:
        print(f"Best model path {config.MODEL_SAVE_PATH} not found for test set. Evaluating with current model state.")

    clip_model.eval()
    final_test_labels_list = []
    final_test_similarity_logits_list = []

    final_test_eval_pbar = tqdm(test_loader, desc="Final Test Evaluation", leave=False)
    with torch.no_grad():
        for batch in final_test_eval_pbar:
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].cpu()

            image_embeds = clip_model.get_image_features(pixel_values=pixel_values)
            text_embeds = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
            image_embeds_norm = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
            text_embeds_norm = text_embeds / text_embeds.norm(dim=-1, keepdim=True) # CORRECTED
            similarity_logits_for_eval = ((image_embeds_norm * text_embeds_norm).sum(dim=1) * clip_model.logit_scale.exp()).cpu()

            final_test_labels_list.append(labels)
            final_test_similarity_logits_list.append(similarity_logits_for_eval)

    if not final_test_labels_list: # Check if test loader was empty or yielded no data
        print("No data processed from test_loader. Skipping test metrics calculation.")
    else:
        final_test_labels_tensor = torch.cat(final_test_labels_list)
        final_test_similarity_logits_tensor = torch.cat(final_test_similarity_logits_list)
        final_test_probs_tensor = torch.sigmoid(final_test_similarity_logits_tensor)

        test_accuracy_metric = Accuracy(task="binary", threshold=config.SIMILARITY_THRESHOLD)
        test_f1_metric = F1Score(task="binary", threshold=config.SIMILARITY_THRESHOLD)
        test_recall_metric = Recall(task="binary", threshold=config.SIMILARITY_THRESHOLD)
        test_auroc_metric = AUROC(task="binary")

        final_test_accuracy = test_accuracy_metric(final_test_probs_tensor, final_test_labels_tensor.int())
        final_test_f1 = test_f1_metric(final_test_probs_tensor, final_test_labels_tensor.int())
        final_test_recall = test_recall_metric(final_test_probs_tensor, final_test_labels_tensor.int())
        final_test_auroc = test_auroc_metric(final_test_similarity_logits_tensor, final_test_labels_tensor.int())

        print(f"\nFinal Test Metrics (Threshold: {config.SIMILARITY_THRESHOLD}):")
        print(f"  Accuracy: {final_test_accuracy:.4f}, F1 Score: {final_test_f1:.4f}, Recall: {final_test_recall:.4f}, ROC AUC: {final_test_auroc:.4f}")

        final_test_preds_binary = (final_test_probs_tensor >= config.SIMILARITY_THRESHOLD).int()
        cm_test_metric = ConfusionMatrix(task="binary", num_classes=2)
        cm_test_tensor = cm_test_metric(final_test_preds_binary, final_test_labels_tensor.int())
        cm_test = cm_test_tensor.cpu().numpy()

        plt.figure(figsize=(7, 6))
        sns.heatmap(cm_test, annot=True, fmt="d", cmap="Greens",
                    xticklabels=["Pred Mismatch (0)", "Pred Match (1)"],
                    yticklabels=["Actual Mismatch (0)", "Actual Match (1)"])
        plt.title(f"Test Set Confusion Matrix (Th: {config.SIMILARITY_THRESHOLD})")
        plt.xlabel("Predicted Label"); plt.ylabel("Actual Label")
        plt.savefig(os.path.join(config.MODEL_SAVE_DIR, "confusion_matrix_test_bce.png")); plt.show()

        y_true_test_np = final_test_labels_tensor.numpy().astype(int)
        y_scores_test_np = final_test_similarity_logits_tensor.numpy()
        fpr_test, tpr_test, _ = sk_roc_curve(y_true_test_np, y_scores_test_np)

        plt.figure(figsize=(7, 6))
        plt.plot(fpr_test, tpr_test, color='purple', lw=2, label=f'Test ROC curve (AUC = {final_test_auroc:.4f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('Test Set ROC Curve')
        plt.legend(loc="lower right"); plt.grid(True)
        plt.savefig(os.path.join(config.MODEL_SAVE_DIR, "roc_curve_test_bce.png")); plt.show()
else:
    print("Skipping final test set evaluation as test_loader was not created (e.g., test_data was empty).")

print(f"\nAll artifacts (model, plots) saved in: {config.MODEL_SAVE_DIR}")
print("--- Script Finished ---")